Ezen notebookban bemutatásra kerül, a GRU modell által prediktált adatok feldolgozása. Ehhez 3 technika van kidolgozva:


1.   Minden modellt követünk. Ha a következő napra magasabb értéket prediktál mint előző napra, akkor veszünk/megtartjuk a részvényt, ha pedig esik akkor eladunk,vagy ha nincs birtokunkban akkor távol maradunk a vételtől.
2.   Arányosan vásároljuk a részvényt, a 10 modellből minél több szerint nőni fog, annál többet veszünk (természetesen ez ugyanúgy igaz eladásnál is).
3.   Csak az eddig legjobban teljesítő K részvényt vesszük figyelmbe.


# 0. Adatelőkészítés

Legelső lépésben az adatok feldolgozása történik, és olyan formába alakítása, mely számunkra megfelelő. Mivel ennek menete a gru_stocks.ipynb-be már megtalálható, így itt ez nincs részletezve.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv(os.path.join('../input/price-volume-data-for-all-us-stocks-etfs/Stocks','gm.us.txt'),delimiter=',',usecols=['Date','Open','High','Low','Close'])
print('Loaded data from the Kaggle repository')

df.head()

In [ ]:
df = df[df['Date'].str.contains("2012") | df['Date'].str.contains("2013") | df['Date'].str.contains("2014") | df['Date'].str.contains("2015") | df['Date'].str.contains("2016") ]
df.head()

In [ ]:
TEST_INTERVAL = 200
TIMESTAMP = 50
NUM_OF_RUNS = 10

In [ ]:
high_prices = df.loc[:,'High'].to_numpy()
low_prices = df.loc[:,'Low'].to_numpy()
mid_prices = (high_prices+low_prices)/2.0

print(mid_prices)

data_num = len(mid_prices)
data_num_train = data_num - TEST_INTERVAL
train_data = mid_prices[:data_num_train]
test_data = mid_prices[data_num_train-TIMESTAMP:]

In [ ]:
sc = MinMaxScaler(feature_range = (0, 1))
train_data = train_data.reshape(-1,1)
train_data_scaled = sc.fit_transform(train_data)

In [ ]:
X_train = []
y_train = []
for i in range(TIMESTAMP, data_num_train):
    X_train.append(train_data_scaled[i-TIMESTAMP:i, 0])
    y_train.append(train_data_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

real_stock_price = test_data[TIMESTAMP:]
predicted_stock_prices = np.zeros((NUM_OF_RUNS,TEST_INTERVAL))

In [ ]:
predicted_stock_prices = np.loadtxt(os.path.join('../input/stock-predictions-gru','gm_pred.csv'), delimiter=",")

In [ ]:
real_stock_price_profit_per_day = []
for i in range(1, TEST_INTERVAL):
    real_stock_price_profit_per_day.append((real_stock_price[i] - real_stock_price[0])/real_stock_price[0] * 100)
    
predicted_stock_price_profits_per_day = [[0 for j in range(TEST_INTERVAL)] for i in range(NUM_OF_RUNS)]

In [ ]:
avg_profit = 0

# 1. technika

Az első technika során minden egyes modell predikcióját feldolgozzuk külön, külön. Ha a következő napra magasabb értéket prediktál a modell mint előző napra, akkor veszünk/megtartjuk a részvényt, ha pedig esik akkor eladunk,(vagy ha nincs birtokunkban akkor távol maradunk a vételtől). 

A következő táblázat mutatja az így kapott eredményeket 11 különböző részvény függvényében:

<a href="https://ibb.co/pyMHLVk"><img src="https://i.ibb.co/tXv0Jj1/image.png" alt="image" border="0"></a>

Látható, hogy 11 részvényből 7 esetben értünk el, jobb eredményt adott idő alatt, mint a részvény valódi növekedése. Tipikusan, azon részvények esetében volt gyengébb a modell predikciója, mint valódi esetben, amikor a részvény rövid idő alatt nagyon nagyott nőtt, viszont veszteség vagy kicsit növekvő részvények esetében, jobb eredményt érhettünk el a modell segítségével, mint a valódi növekedés. 

A következő ábra szemlélteti egy részvény, nevezetesen a Netflix, növekedését, és az egyes modellek profitjait külön-külön:

<a href="https://imgbb.com/"><img src="https://i.ibb.co/3RmYHT8/image.png" alt="image" border="0"></a>

A fekete vonal reprezentálja a részvény valódi növekedését. Itt is látható, hogy amíg a részvény veszteséges, minden predikció jobban teljesít a részvény valódi profitjához képest, ugyanakkor a hirtelen növekedés után több modell is az eladás/tartózkodás mellett dönt, így végső soron a 10 modell közül 3 rosszabbul teljesít, mint a valódi növekedés, és 7 pedig jobban. 

Végső soron, a fenti adatok arra engednek következtetni, hogy a modellünk segítségével stabil, kevésbé volatilis részvényekbe való befektetéssel, jobb eredményeket is elérhetünk, mint a valódi növekedés.

In [ ]:
def calculateProfit(diffStock):
    all_profit = 0
    min_profit = 10000000
    max_profit = -10000000
    for j in range(NUM_OF_RUNS):
        print(j, " RUN")
        predicted_stock_price = predicted_stock_prices[j]
        original_stock_price = real_stock_price[0]

        bought_stock_price = real_stock_price[0]
        stock_price_at_selling = 0
        stock_buys_total = real_stock_price[0]
        stock_sells_total = 0
        profit = 0

        for i in range(1, TEST_INTERVAL):
            if(predicted_stock_price[i]>diffStock[i-1]):
                if(bought_stock_price==0):
                  print("Buying at ", i, " day:", real_stock_price[i])
                  bought_stock_price = real_stock_price[i]
                  stock_buys_total = stock_buys_total + real_stock_price[i]
            else:
                if(bought_stock_price!=0):
                  print("Selling at ", i, " day:", real_stock_price[i], "     We made:", real_stock_price[i]-bought_stock_price)
                  bought_stock_price = 0
                  stock_price_at_selling = real_stock_price[i]
                  stock_sells_total = stock_sells_total + real_stock_price[i]
            profit = stock_sells_total - stock_buys_total
            if(bought_stock_price==0):
                predicted_stock_price_profits_per_day[j][i] = (profit)/original_stock_price * 100
            else:
                predicted_stock_price_profits_per_day[j][i] = (profit+real_stock_price[i])/original_stock_price * 100

        print("Money spent on buying:", stock_buys_total)    
        print("Money received on selling:", stock_sells_total) 
        print("cash profit:", profit)
        print("actual stock price:", real_stock_price[TEST_INTERVAL-1]) 

        if(bought_stock_price==0):
            print("We made ", profit, " from ", original_stock_price)
            act_profit = (profit)/original_stock_price * 100
            print("Profit:", act_profit, "%")
            all_profit = all_profit + act_profit
            if(act_profit>max_profit):
                max_profit = act_profit
            if(act_profit<min_profit):
                min_profit = act_profit
            print("Stock growth:", real_stock_price[TEST_INTERVAL-1], " ", (real_stock_price[TEST_INTERVAL-1] - original_stock_price)/original_stock_price * 100, "%")
        else:
            print("We made ", profit+real_stock_price[TEST_INTERVAL-1], " from ", original_stock_price)
            act_profit = (profit+real_stock_price[TEST_INTERVAL-1])/original_stock_price * 100
            print("Profit:", (profit+real_stock_price[TEST_INTERVAL-1])/original_stock_price * 100, "%")
            all_profit = all_profit + act_profit
            if(act_profit>max_profit):
                max_profit = act_profit
            if(act_profit<min_profit):
                min_profit = act_profit
            print("Stock growth:", real_stock_price[TEST_INTERVAL-1], " ", (real_stock_price[TEST_INTERVAL-1] - original_stock_price)/original_stock_price * 100, "%")
        print(" ")
        print(" ")
        print(" ")
        print(" ")
        print(" ")
        print(" ")
    avg_profit = all_profit / NUM_OF_RUNS 
    
    print("AVG profit:", avg_profit)
    print("MAX profit:", max_profit)
    print("MIN profit:", min_profit)

# 2. technika

A második technika során a modellek együttes predikcióját vesszük figyelmbe, azaz azt, hogy a 10 modell közül, hány szerint fog nőni és hány szerint fog csökkeni másnapra a részvény értéke. Ehhez a következő metrikákat határoztam meg:


*   SUPER_STRONG_SELL : Ha 0 vagy 1 modell prediktálja a növekedést, akkor nagyon nagy mértékű eladást végzünk
*   STRONG_SELL : Ha 2 vagy 3 modell prediktálja a növekedést akkor nagy mértékű eladást végzünk
*   SELL : Ha 4 modell prediktálja a növekedést, akkor eladást végzünk
*   STALL : Ha fele-fele arányban van prediktálva a növekedés és csökkenés, akkor nem csinálunk semmit
*   BUY : Ha 6 modell prediktálja a növekedést, akkor vásárolunk
*   STRONG_BUY : Ha 7 vagy 8, akkor nagy mértékben vásárolunk
*   SUPER_STRONG_BUY : Ha 9 vagy 10, akkor nagyon nagy mértékben vásárolunk

A sima, STRONG és SUPER_STRONG következő értékeire van részletesen dokumentálva az eredmény : 0.03/0.06/0.10 (kisebb arányú változtatás), 0.01/0.05/0.20 (nagyobb arányú). Kikötjük ezen kívúl, hogy egy részvényből legfeljebb 1 egységünk lehet, s eladásnál nem mehetünk 0 alá (shortolást nem végzünk). Az alábbi táblázat mutatja az így kapott eredményeket:

<a href="https://ibb.co/PNhCQSW"><img src="https://i.ibb.co/hyW8fqd/image.png" alt="image" border="0"></a>

Látható, hogy a 11 részvényből 8 eseténél profitot sikerült elérni a nagyobb mértékű arányosítással (0.01, 0.05, 0.20) a valódi növekedéshez képest, míg a gyengébb arányosításnál 7 esetben sikerült a profit (Netflix a kivétel). Ugyanakkor a gyengébb arányosítással (0.03/0.06/0.10) 7 esetben jobb eredményt értünk el, mint az erős arányosítással, és csak 4 esetben rosszabbat. 

Végső soron, további elemzésre, optimalizálásra lehet szükség, ahhoz, hogy el tudjuk dönteni, hogy mik a legoptimálisabb arányok. Ugyanakkor látható, hogy nagyobb profitot tudtunk itt elérni, mint az első esetben.



In [ ]:
def calculateProfitUsingAll(diffStock, NORMAL_BUY_SELL, STRONG_BUY_SELL, SUPER_STRONG_BUY_SELL):
    STOCK_CAP = 100
    SUPER_STRONG_SELL_MIN = 0
    STRONG_SELL_MIN = 2
    SELL_MIN = 4
    STALL_MIN = 5
    BUY_MIN = 6
    STRONG_BUY_MIN = 7
    SUPER_STRONG_BUY_MIN = 9
    
    bought_stocks = STOCK_CAP//4
    money_spent = bought_stocks * real_stock_price[0]
    money_received = 0
    profits_per_day = []
    for i in range(1, TEST_INTERVAL):
        BUY_PRED_NUM = 0
        for j in range(NUM_OF_RUNS):
            predicted_stock_price = predicted_stock_prices[j][i]
            if(predicted_stock_price>diffStock[i-1]):
                BUY_PRED_NUM = BUY_PRED_NUM + 1
        if(BUY_PRED_NUM<STRONG_SELL_MIN):
            if(bought_stocks > 0):
                if(bought_stocks > SUPER_STRONG_BUY_SELL):
                    bought_stocks = bought_stocks - SUPER_STRONG_BUY_SELL
                    money_received = money_received + SUPER_STRONG_BUY_SELL * real_stock_price[i]
                    print(i, ". day: SUPER STRONG SELL. We sold ", SUPER_STRONG_BUY_SELL, " stocks. Current stock number:", bought_stocks)
                else:
                    bs_temp = bought_stocks
                    bought_stocks = bought_stocks - bs_temp
                    money_received = money_received + bs_temp * real_stock_price[i]
                    print(i, ". day: SUPER STRONG SELL. We sold ", bs_temp, " stocks. Current stock number:", bought_stocks)
        elif(BUY_PRED_NUM<SELL_MIN):
            if(bought_stocks > 0):
                if(bought_stocks > STRONG_BUY_SELL):
                    bought_stocks = bought_stocks - STRONG_BUY_SELL
                    money_received = money_received + STRONG_BUY_SELL * real_stock_price[i]
                    print(i, ". day: STRONG SELL. We sold ", STRONG_BUY_SELL, " stocks. Current stock number:", bought_stocks)
                else:
                    bs_temp = bought_stocks
                    bought_stocks = bought_stocks - bs_temp
                    money_received = money_received + bs_temp * real_stock_price[i]
                    print(i, ". day: STRONG SELL. We sold ", bs_temp, " stocks. Current stock number:", bought_stocks)
        elif(BUY_PRED_NUM<STALL_MIN):
            if(bought_stocks > 0):
                bought_stocks = bought_stocks - NORMAL_BUY_SELL
                money_received = money_received + NORMAL_BUY_SELL * real_stock_price[i]
                print(i, ". day: SELL. We sold ", NORMAL_BUY_SELL," stock. Current stock number:", bought_stocks)
        elif(BUY_PRED_NUM<BUY_MIN):
            print(i, ". day: STALL. Current stock number:", bought_stocks)
        elif(BUY_PRED_NUM<STRONG_BUY_MIN):
            if(bought_stocks < STOCK_CAP):
                bought_stocks = bought_stocks + NORMAL_BUY_SELL
                money_spent = money_spent + NORMAL_BUY_SELL * real_stock_price[i]
                print(i, ". day: BUY. We bought ", NORMAL_BUY_SELL," stock. Current stock number:", bought_stocks)
        elif(BUY_PRED_NUM<SUPER_STRONG_BUY_MIN):
            if(bought_stocks < STOCK_CAP):
                stocks_to_cap = STOCK_CAP - bought_stocks
                if(stocks_to_cap < STRONG_BUY_SELL):
                    bought_stocks = bought_stocks + stocks_to_cap
                    money_spent = money_spent + stocks_to_cap * real_stock_price[i]
                    print(i, ". day: BUY. We bought ", stocks_to_cap, " stock. Current stock number:", bought_stocks)
                else:
                    bought_stocks = bought_stocks + STRONG_BUY_SELL
                    money_spent = money_spent + STRONG_BUY_SELL * real_stock_price[i]
                    print(i, ". day: STRONG BUY. We bought", STRONG_BUY_SELL, " stock. Current stock number:", bought_stocks)
        else:
            if(bought_stocks < STOCK_CAP):
                stocks_to_cap = STOCK_CAP - bought_stocks
                if(stocks_to_cap < SUPER_STRONG_BUY_SELL):
                    bought_stocks = bought_stocks + stocks_to_cap
                    money_spent = money_spent + stocks_to_cap * real_stock_price[i]
                    print(i, ". day: BUY. We bought ", stocks_to_cap, " stock. Current stock number:", bought_stocks)
                else:
                    bought_stocks = bought_stocks + SUPER_STRONG_BUY_SELL
                    money_spent = money_spent + SUPER_STRONG_BUY_SELL * real_stock_price[i]
                    print(i, ". day: SUPER STRONG BUY. We bought", SUPER_STRONG_BUY_SELL, " stock. Current stock number:", bought_stocks) 
        profits_per_day.append((((real_stock_price[i]*bought_stocks + money_received)/money_spent) - 1)*100)
    print("Money spent on buying:", money_spent)
    print("Money received on selling:", money_received)
    print("Current money in our stocks:", real_stock_price[TEST_INTERVAL-1]*bought_stocks)
    print("Total profit:", (((real_stock_price[TEST_INTERVAL-1]*bought_stocks + money_received - money_spent)/(STOCK_CAP*real_stock_price[0])))*100, "%")
    return profits_per_day
            

# 3. technika

A harmadik technika alapvetése az, hogy amelyik modell eddig jó profitot hozott, az a továbbiakban is nyereséges lesz. Ezért, nem szükséges az összes modellt figyelni, elég csupán azokat, amelyek eddig is a legjobb profitot hozták. 

Ez a technika 2 esetre van dokumentálva: ha csak a legjobb modeltt vesszük, valamint ha a 3 legjobbat vesszük, és ha legalább 2 növekedést/csökkenést mond akkor veszünk/eladunk. Az így kapott eredményeket az alábbi táblázáat mutatja:

<a href="https://ibb.co/jHGrJ9d"><img src="https://i.ibb.co/x3G7FKy/image.png" alt="image" border="0"></a>

Látható, hogy 11 esetből 9szer nyereségek voltunk a valódi növekedéshez képest TOP1 esetben, és 8szor TOP3 esetben (Apple a kivétel). Ezen kívűl 7szer jobban teljesített a TOP1, mint a TOP3, így azt is elmondhatjuk, hogy ezen a teszthalmazon jobban teljesített a TOP1 mint a TOP3.

Végső soron, azt láthatjuk be, hogy a 3. technika teljesített a legjobban, ezen belül pedig az az eset, amikor csak az eddigi legjobb modellt vesszük figyelembe.

In [ ]:
def calculateProfitTopN(diffStock, N):
    money_spent = real_stock_price[0]
    money_received = 0
    bought_stocks = 1
    bought_stock_price = real_stock_price[0]
    profit = 0
    original_stock_price = real_stock_price[0]
    for i in range(1, TEST_INTERVAL):
        np_predicted_stock_price_profits_per_day = np.array(predicted_stock_price_profits_per_day)
        profits_per_run = np_predicted_stock_price_profits_per_day[:,i]
        profits_per_run_idxs = np.argsort(profits_per_run)[::-1]
        best_idxs = profits_per_run_idxs[:N]
        should_buy = 0
        for j in best_idxs:
            predicted_stock_price = predicted_stock_prices[j]
            if(predicted_stock_price[i]>diffStock[i-1]):
                should_buy = should_buy + 1
            else:
                should_buy = should_buy - 1
        if(bought_stocks<1 and should_buy>0):
            print("Buying at ", i, " day:", real_stock_price[i])
            bought_stocks = bought_stocks + 1
            money_spent = money_spent + real_stock_price[i]
            bought_stock_price = real_stock_price[i]
        elif(bought_stocks>0 and not should_buy>0):
            print("Selling at ", i, " day:", real_stock_price[i], "     We made:", real_stock_price[i]-bought_stock_price)
            bought_stocks = bought_stocks - 1
            money_received = money_received + real_stock_price[i]
        profit = money_received - money_spent
    print("Money spent on buying:", money_spent)
    print("Money received on selling:", money_received)
    print("Current money in our stocks:", real_stock_price[TEST_INTERVAL-1]*bought_stocks)
    print("cash profit:", profit)
    if(bought_stocks==0):
            print("We made ", profit, " from ", original_stock_price)
            act_profit = (profit)/original_stock_price * 100
            print("Profit:", act_profit, "%")
    else:
            print("We made ", profit+real_stock_price[TEST_INTERVAL-1], " from ", original_stock_price)
            act_profit = (profit+real_stock_price[TEST_INTERVAL-1])/original_stock_price * 100
            print("Profit:", (profit+real_stock_price[TEST_INTERVAL-1])/original_stock_price * 100, "%")
       

In [ ]:
print("Comparing to real stock price:")
calculateProfit(real_stock_price)

In [ ]:
profits_per_day_by_all = []
profits_per_day_by_all = calculateProfitUsingAll(real_stock_price, 1, 5, 20)

In [ ]:
profits_per_day_by_all = calculateProfitUsingAll(real_stock_price, 3, 6, 10)

In [ ]:
calculateProfitTopN(real_stock_price, 1)

In [ ]:
plt.figure(figsize = (18,9))
for i in range(NUM_OF_RUNS):
    plt.plot(predicted_stock_price_profits_per_day[i], color = np.random.rand(3,)
             , label = 'Stock Price Profit Prediction ' + str(i))
plt.plot(real_stock_price_profit_per_day, color = 'black', label = 'Stock Price Profit')
#plt.plot(profits_per_day_by_all, color = 'red', label = 'Profits by using all')
plt.title('Stock Price Profit Per Day')
plt.xlabel('Time')
plt.ylabel('Profit')
plt.legend()
plt.show()

In [ ]:
calculateProfitUsingAll(real_stock_price)